## Web Scrapping

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By

import requests
from bs4 import BeautifulSoup

import pandas as pd

In [9]:
titles = []
articles = []
categories = []


## Site the conversation

In [95]:
url = 'https://theconversation.com/africa'

In [96]:
response = requests.get(url)
links = []
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    navbar = soup.body.find_all('nav',{'class':'small-sidebar-group sections-sidebar-group'})
    for nav in navbar:
        l= nav.find_all('a')
    links = [i['href'] for i in l]


In [97]:
links

[]

In [93]:
article_links = []
url = 'https://theconversation.com'
for link in links:
    path = url+link
    response = requests.get(path)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        for s in soup.body.find_all('h2'):
            article_links.append(s.a['href'])
    

In [94]:
len(article_links)

0

In [15]:
driver = webdriver.Chrome()

In [16]:
for a in article_links:
    path = url+a
    driver.get(path)
    titre = driver.find_elements(By.TAG_NAME,'h1')
    titles.append(titre[0].text)
    contenu = driver.find_elements(By.XPATH,'//*[@itemprop="articleBody"]')
    articles.append(contenu[0].text)
    response = requests.get(path)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        try:
            categories.append(soup.body.find_all('a',{'class':'active'})[0].text.lower())
        except:
            categories.append('Null')
    
    

In [17]:
len(titles)

443

In [18]:
len(articles)

443

In [19]:
len(categories)

443

In [20]:
data={'article':articles,'titre':titles,'categorie':categories}

In [21]:
df = pd.DataFrame(data)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 443 entries, 0 to 442
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   article    443 non-null    object
 1   titre      443 non-null    object
 2   categorie  443 non-null    object
dtypes: object(3)
memory usage: 10.5+ KB


In [24]:
len(df.article.unique())

379

In [26]:
df.categorie.value_counts()

politics                71
environment + energy    67
arts + culture          65
science + tech          57
business + economy      49
education               43
Null                    39
health + medicine       37
home page               15
Name: categorie, dtype: int64

In [27]:
df.to_csv('Conversation_data.csv',index= False)